## The Monty Hall Problem

Here's a fun and perhaps surprising statistical riddle, and a good way to get some practice writing python functions

In a gameshow, contestants try to guess which of 3 closed doors contain a cash prize (goats are behind the other two doors). Of course, the odds of choosing the correct door are 1 in 3. As a twist, the host of the show occasionally opens a door after a contestant makes his or her choice. This door is always one of the two the contestant did not pick, and is also always one of the goat doors (note that it is always possible to do this, since there are two goat doors). At this point, the contestant has the option of keeping his or her original choice, or swtiching to the other unopened door. The question is: is there any benefit to switching doors? The answer surprises many people who haven't heard the question before.

We can answer the problem by running simulations in Python. We'll do it in several parts.

First, write a function called simulate_prizedoor. This function will simulate the location of the prize in many games -- see the detailed specification below:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
"""
Function
--------
simulate_prizedoor

Generate a random array of 0s, 1s, and 2s, representing
hiding a prize between door 0, door 1, and door 2

Parameters
----------
nsim : int
    The number of simulations to run

Returns
-------
sims : array
    Random array of 0s, 1s, and 2s

Example
-------
>>> print (simulate_prizedoor(3))
array([0, 0, 2])
"""
def simulate_prizedoor(nsim):
    #compute here
    answer = np.random.randint(0, 3, size=nsim)
    return answer

Next, write a function that simulates the contestant's guesses for nsim simulations. Call this function simulate_guess. The specs:

In [3]:
"""
Function
--------
simulate_guess

Return any strategy for guessing which door a prize is behind. This
could be a random strategy, one that always guesses 2, whatever.

Parameters
----------
nsim : int
    The number of simulations to generate guesses for

Returns
-------
guesses : array
    An array of guesses. Each guess is a 0, 1, or 2

Example
-------
>>> print (simulate_guess(5))
array([0, 0, 0, 0, 0])
"""
#your code here
def simulate_guess(nsim):
    return np.zeros(nsim, dtype=int)

Next, write a function, goat_door, to simulate randomly revealing one of the goat doors that a contestant didn't pick.

In [4]:
"""
Function
--------
goat_door

Simulate the opening of a "goat door" that doesn't contain the prize,
and is different from the contestants guess

Parameters
----------
prizedoors : array
    The door that the prize is behind in each simulation
guesses : array
    THe door that the contestant guessed in each simulation

Returns
-------
goats : array
    The goat door that is opened for each simulation. Each item is 0, 1, or 2, and is different
    from both prizedoors and guesses

Examples
--------
>>> print (goat_door(np.array([0, 1, 2]), np.array([1, 1, 1])))
>>> array([2, 2, 0])
"""
#your code here
def goat_door(guesses, prizedoors):
    result = np.random.randint(0, 3, prizedoors.size)
    while True:
        bad = (result == prizedoors) | (result == guesses)
        if not bad.any():
            return result
        result[bad] = np.random.randint(0, 3, bad.sum())

Write a function, switch_guess, that represents the strategy of always switching a guess after the goat door is opened.

In [5]:
"""
Function
--------
switch_guess

The strategy that always switches a guess after the goat door is opened

Parameters
----------
guesses : array
     Array of original guesses, for each simulation
goatdoors : array
     Array of revealed goat doors for each simulation

Returns
-------
The new door after switching. Should be different from both guesses and goatdoors

Examples
--------
>>> print (switch_guess(np.array([0, 1, 2]), np.array([1, 2, 1])))
>>> array([2, 0, 0])
"""
#your code here
def switch_guess(guesses, goatdoors):
    result = np.random.randint(0, 3, guesses.size)
    while True:
        bad = (result == guesses) | (result == goatdoors)
        if not bad.any():
            return result
        result[bad] = np.random.randint(0, 3, bad.sum())
        
# Alternate method
# def switch_guess(guesses, goatdoors):
#     result = np.zeros(guesses.size)
#     switch = {(0, 1): 2, (0, 2): 1, (1, 0): 2, (1, 2): 1, (2, 0): 1, (2, 1): 0}
#     for i in [0, 1, 2]:
#         for j in [0, 1, 2]:
#             mask = (guesses == i) & (goatdoors == j)
#             if not mask.any():
#                 continue
#             result = np.where(mask, np.ones_like(result) * switch[(i, j)], result)
#     return result

Last function: write a win_percentage function that takes an array of guesses and prizedoors, and returns the percent of correct guesses

In [6]:
"""
Function
--------
win_percentage

Calculate the percent of times that a simulation of guesses is correct

Parameters
-----------
guesses : array
    Guesses for each simulation
prizedoors : array
    Location of prize for each simulation

Returns
--------
percentage : number between 0 and 100
    The win percentage

Examples
---------
>>> print (win_percentage(np.array([0, 1, 2]), np.array([0, 0, 0])))
33.333
"""
#your code here
def win_percentage(guesses, prizedoors):
    return 100 * (guesses == prizedoors).mean()

Now, put it together. Simulate 10000 games where contestant keeps his original guess, and 10000 games where the contestant switches his door after a goat door is revealed. Compute the percentage of time the contestant wins under either strategy. Is one strategy better than the other?

In [7]:
#your code here

nsim = 10000

#keep guesses
print ("Win percentage when keeping original door")
print(win_percentage(simulate_guess(nsim), simulate_prizedoor(nsim)))

#switch

initial_guesses = simulate_guess(nsim)
prize_doors = simulate_prizedoor(nsim)
goats = goat_door(initial_guesses, prize_doors)
final_guesses = switch_guess(initial_guesses, goats)
print ("Win percentage when switching doors")
print(win_percentage(final_guesses, prize_doors))

Win percentage when keeping original door
33.43
Win percentage when switching doors
66.96


Many people find this answer counter-intuitive (famously, PhD mathematicians have incorrectly claimed the result must be wrong. Clearly, none of them knew Python).

One of the best ways to build intuition about why opening a Goat door affects the odds is to re-run the experiment with 100 doors and one prize. If the game show host opens 98 goat doors after you make your initial selection, would you want to keep your first pick or switch? Can you generalize your simulation code to handle the case of n doors?

In [8]:
def one_round(doors, switch):
    prize_door = np.random.randint(0, doors)
    initial_guess = np.random.randint(0, doors)
    
    if initial_guess != prize_door:
        door_not_open = prize_door
    else:
        while True:
            door_not_open = np.random.randint(0, doors)
            if door_not_open != initial_guess:
                break
    if switch:
        final_guess = door_not_open
    else:
        final_guess = initial_guess
        
    win = final_guess == prize_door
    
    # print(initial_guess == prize_door)
    return win

In [9]:
def simulation(iterations, doors):
    games_won_switch = 0
    games_won_noswitch = 0
    for i in range(iterations):
        win = one_round(doors, False)
        if win:   
            games_won_noswitch = games_won_noswitch + 1
        win = one_round(doors, True)
        if win:
            games_won_switch += 1
    
    switching_percentage = games_won_switch/iterations*100
    non_switching_percentage = games_won_noswitch/iterations*100
    print("Monty Hall Simulation with {} doors".format(doors))
    print('Iterations: {}'.format(iterations))
    print('winning percentage when switching doors: {}%'.format(switching_percentage))
    print('winning percentage when not switching doors: {}%'.format(non_switching_percentage))

In [10]:
# Play around with the number of iterations and the doors parameters
simulation(iterations=1000, doors=10)

Monty Hall Simulation with 10 doors
Iterations: 1000
winning percentage when switching doors: 89.7%
winning percentage when not switching doors: 8.5%
